## 一、图的重要概念（看PPT）
#### (1) 顶点、边 
#### (2) 无向图、有向图
#### (3) 度、入度、出度
#### (4) 连通图、强连通图
#### (5) 连通分量、强连通分量
#### (6) 简单图、多重图、伪图 

## 二、使用GraphFrames库
#### 1. GraphFrames库目前还没有官方并入Spark项目，因此需要自己加载该库
#### 2. 加载时注意版本匹配问题
##### (1) 首先查看spark所用的scala版本：在cmd中运行spark-shell命令或者在 localhost:4040 中查看
##### (2) https://spark-packages.org/package/graphframes/graphframes 中找到对应的GraphFrames版本


In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder \
                    .config('spark.jars.packages', 'graphframes:graphframes:0.8.2-spark3.0-s_2.12') \
                    .getOrCreate()
# 上面代码会自动下载graphframes

In [2]:
spark

## 三、创建图实例
#### (1) 创建顶点DataFrame对象：必须有id列，作为顶点的唯一标识
#### (2) 创建边DataFrame对象：必须有src和dst列
#### (3) 根据顶点和边DataFrame实例，创建图实例

<img src = 'graph.png' align = left>

In [3]:
# 创建顶点DataFrame实例
data_V = [('a', 'Alice', 34), ('b', 'Bob', 36), ('c', 'Charlie', 30), ('d', 'David', 29), ('e', 'Esther', 32), ('f', 'Fanny', 36), ('g', 'Gabby', 60)]
cols_V = ['id', 'name', 'age']
vertices = spark.createDataFrame(data_V, cols_V)
vertices.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  a|  Alice| 34|
|  b|    Bob| 36|
|  c|Charlie| 30|
|  d|  David| 29|
|  e| Esther| 32|
|  f|  Fanny| 36|
|  g|  Gabby| 60|
+---+-------+---+



In [4]:
filterV = vertices.filter('age<30')
filterV.show()

+---+-----+---+
| id| name|age|
+---+-----+---+
|  d|David| 29|
+---+-----+---+



In [5]:
# 创建边DataFrame实例
data_E = [('a', 'b', 'friend'), ('b', 'c', 'follow'), ('c', 'b', 'follow'), ('f', 'c', 'follow'), ('e', 'f', 'follow'), ('e', 'd', 'friend'), 
         ('d', 'a', 'friend'), ('a', 'e', 'friend')]
cols_E = ['src', 'dst', 'relationship']
edges = spark.createDataFrame(data_E, cols_E)
edges.show()

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  a|  b|      friend|
|  b|  c|      follow|
|  c|  b|      follow|
|  f|  c|      follow|
|  e|  f|      follow|
|  e|  d|      friend|
|  d|  a|      friend|
|  a|  e|      friend|
+---+---+------------+



In [6]:
followEdges = edges.filter("relationship = 'follow'")
followEdges.show()

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  b|  c|      follow|
|  c|  b|      follow|
|  f|  c|      follow|
|  e|  f|      follow|
+---+---+------------+



In [7]:
# 创建图对象
from graphframes import GraphFrame
graph = GraphFrame(vertices, edges)

## 四、根据图对象，查看顶点DataFrame（顶点视图）、边DataFrame（边视图）、三元组视图、模式视图、出入度
#### 1. 查看顶点DataFrame（顶点视图)

In [ ]:
type(graph.vertices)

In [ ]:
graph.vertices.show()

#### 2. 查看边DataFrame（边视图）

In [ ]:
graph.edges.show()

#### 3. 查看三元组视图

In [ ]:
graph.triplets.show()

#### 4. 查看模式视图

In [ ]:
m_df = graph.find("(v1)-[e1]->(v2)")
m_df.show()

In [ ]:
m_df = graph.find("(v1)-[]->(v2)")
m_df.show()

In [ ]:
m_df.groupBy('v1').count().alias('count').orderBy('count', ascending = False).show()

In [ ]:
m_df.groupBy('v2').count().alias('count').orderBy('count', ascending = False).show()

In [ ]:
m_df = graph.find("(v1)-[]->(v2);(v2) - [] -> (v1)")   # 相互关注
m_df.show()

In [ ]:
m_df = graph.find("(v1)-[]->(v2);!(v2) - [] -> (v1)")  # 单向关注
m_df.show()

In [ ]:
m_df.printSchema()

In [ ]:
from pyspark.sql.functions import col
m_df.select(col('v1.*'),col('v2.*')).show()

In [ ]:
m_df.select(col('v1.id').alias('v1_id'),col('v2.name').alias('v2_name')).show()

In [ ]:
m_df.select(col('v1.id').alias('v1_id')).where(col('v2.id') == 'a').show()

#### 5. 查看度、出度、入度

In [ ]:
graph.degrees.show()

In [ ]:
graph.inDegrees.show()

In [ ]:
graph.inDegrees.orderBy('inDegree', ascending = False).show()

In [ ]:
graph.outDegrees.show()

In [ ]:
graph.outDegrees.orderBy(['outDegree', 'id'], ascending = False).show()

## 五、算法
#### 1. 广度优先搜索(Breadth-First Search, BFS)：只返回所有匹配路径中的最短路径
#### bfs(fromExpr, toExpr, edgeFilter = None, maxPathLength = 10)

In [ ]:
paths = graph.bfs("name='Esther'", "age <=40", maxPathLength=2)
paths.show()

In [ ]:
paths = graph.bfs("name='Esther'", "age <= 30", maxPathLength=5)
paths.show()

In [ ]:
paths = graph.bfs("name='Esther'", "age == 30", maxPathLength=5)
paths.show()

In [ ]:
paths = graph.bfs("name = 'Esther'", "age > 32", edgeFilter="relationship != 'follow'", maxPathLength=5)
paths.show()

#### 2. 最短路径算法（Dijkstra算法）：返回所有顶点到目标顶点的最短距离
#### shortestPaths(landmarks)

In [ ]:
df = graph.shortestPaths(['g'])
df.show()

In [ ]:
df.printSchema()

In [ ]:
from pyspark.sql.functions import explode, explode_outer
df.select('id', 'name', 'age',explode('distances')).show()

In [ ]:
df.select('id', 'name', 'age',explode_outer('distances')).show()

In [ ]:
df.select('id', 'name', 'age',explode('distances')).filter("value > 0").show()

In [ ]:
df = graph.shortestPaths(['a', 'd'])
df.show()

In [ ]:
df.select('id', 'name', 'age', explode('distances')).show()

In [ ]:
df.select('id', 'name', 'age', explode('distances')).filter("value > 0 and value <= 1").show()

#### 3. 三角形计数算法：确定每个顶点的三角形数量。图被当做无向图处理，平行边计算一次，自环被忽略。三角形个数越多，说明网络顶点之间的连接越紧密
#### triangleCount()

In [8]:
graph.triangleCount().orderBy('count', ascending=False).show()

+-----+---+-------+---+
|count| id|   name|age|
+-----+---+-------+---+
|    1|  e| Esther| 32|
|    1|  a|  Alice| 34|
|    1|  d|  David| 29|
|    0|  g|  Gabby| 60|
|    0|  b|    Bob| 36|
|    0|  c|Charlie| 30|
|    0|  f|  Fanny| 36|
+-----+---+-------+---+



#### 4. 发现图中的环。经常用于社交网络分析，发现社交圈子

####  (1) 强连通分量算法：把图作为有向图
#### stronglyConnectedComponents

In [9]:
graph.stronglyConnectedComponents(maxIter=1).orderBy('component').show()

+---+-------+---+-------------+
| id|   name|age|    component|
+---+-------+---+-------------+
|  g|  Gabby| 60| 146028888064|
|  f|  Fanny| 36| 412316860416|
|  e| Esther| 32| 670014898176|
|  d|  David| 29| 807453851648|
|  c|Charlie| 30|1047972020224|
|  b|    Bob| 36|1382979469312|
|  a|  Alice| 34|1460288880640|
+---+-------+---+-------------+



In [10]:
graph.stronglyConnectedComponents(maxIter=2).orderBy('component').show()

+---+-------+---+-------------+
| id|   name|age|    component|
+---+-------+---+-------------+
|  g|  Gabby| 60| 146028888064|
|  f|  Fanny| 36| 412316860416|
|  a|  Alice| 34| 670014898176|
|  e| Esther| 32| 670014898176|
|  d|  David| 29| 670014898176|
|  c|Charlie| 30|1047972020224|
|  b|    Bob| 36|1382979469312|
+---+-------+---+-------------+



In [11]:
graph.stronglyConnectedComponents(maxIter=3).orderBy('component').show()

+---+-------+---+-------------+
| id|   name|age|    component|
+---+-------+---+-------------+
|  g|  Gabby| 60| 146028888064|
|  f|  Fanny| 36| 412316860416|
|  d|  David| 29| 670014898176|
|  a|  Alice| 34| 670014898176|
|  e| Esther| 32| 670014898176|
|  c|Charlie| 30|1047972020224|
|  b|    Bob| 36|1047972020224|
+---+-------+---+-------------+



In [12]:
graph.stronglyConnectedComponents(maxIter=4).orderBy('component').show()

+---+-------+---+-------------+
| id|   name|age|    component|
+---+-------+---+-------------+
|  g|  Gabby| 60| 146028888064|
|  f|  Fanny| 36| 412316860416|
|  e| Esther| 32| 670014898176|
|  a|  Alice| 34| 670014898176|
|  d|  David| 29| 670014898176|
|  b|    Bob| 36|1047972020224|
|  c|Charlie| 30|1047972020224|
+---+-------+---+-------------+



####  (2)连通分量算法：把图作为无向图
#### connectedComponents()

In [ ]:
# 使用连通分量算法，需要首先设置检查点目录
spark.sparkContext.setCheckpointDir('./checkpointDir')    
df = graph.connectedComponents()                 # 连通分量（最大连接子图）
df.show()   

In [ ]:
from pyspark.sql.functions import collect_list
df_agg = df.groupBy('component').agg(collect_list('name').alias('all_name'))
df_agg.show()

In [ ]:
df_agg.show(truncate=False)    # 列内容的显示不被截断

In [ ]:
from pyspark.sql.functions import concat_ws
df_string = df.groupBy('component').agg(concat_ws('+', collect_list('name')).alias('all_name'))
df_string.show(truncate=False)

#### 5. 标签传播算法（Label Propagation Algorithm, LPA）：把图作为无向图，经常用于社区发现
#### labelPropagation(maxIter)

<img src ='LPA.png' align = left>

In [13]:
graph.labelPropagation(maxIter=1).show()

+---+-------+---+-------------+
| id|   name|age|        label|
+---+-------+---+-------------+
|  g|  Gabby| 60| 146028888064|
|  b|    Bob| 36|1047972020224|
|  e| Esther| 32|1460288880640|
|  a|  Alice| 34|1382979469312|
|  f|  Fanny| 36|1047972020224|
|  d|  David| 29|1460288880640|
|  c|Charlie| 30|1382979469312|
+---+-------+---+-------------+



In [14]:
graph.labelPropagation(maxIter=2).show()

+---+-------+---+-------------+
| id|   name|age|        label|
+---+-------+---+-------------+
|  g|  Gabby| 60| 146028888064|
|  b|    Bob| 36|1382979469312|
|  e| Esther| 32|1460288880640|
|  a|  Alice| 34|1460288880640|
|  f|  Fanny| 36|1382979469312|
|  d|  David| 29|1382979469312|
|  c|Charlie| 30|1047972020224|
+---+-------+---+-------------+



In [15]:
graph.labelPropagation(maxIter=3).show()

+---+-------+---+-------------+
| id|   name|age|        label|
+---+-------+---+-------------+
|  g|  Gabby| 60| 146028888064|
|  b|    Bob| 36|1047972020224|
|  e| Esther| 32|1382979469312|
|  a|  Alice| 34|1382979469312|
|  f|  Fanny| 36|1460288880640|
|  d|  David| 29|1460288880640|
|  c|Charlie| 30|1382979469312|
+---+-------+---+-------------+



In [16]:
graph.labelPropagation(maxIter=4).show()

+---+-------+---+-------------+
| id|   name|age|        label|
+---+-------+---+-------------+
|  g|  Gabby| 60| 146028888064|
|  b|    Bob| 36|1382979469312|
|  e| Esther| 32|1460288880640|
|  a|  Alice| 34|1460288880640|
|  f|  Fanny| 36|1382979469312|
|  d|  David| 29|1382979469312|
|  c|Charlie| 30|1047972020224|
+---+-------+---+-------------+



#### 6. PageRank算法：评估有向图中各个顶点的重要程度（被指向的次数）
#### pageRank(maxIter)
#### (1) 静态PageRank, 通过指定maxIter参数，迭代固定的次数

In [17]:
graph.pageRank(maxIter=5).vertices.orderBy('pagerank').show()    # 迭代5次

+---+-------+---+-------------------+
| id|   name|age|           pagerank|
+---+-------+---+-------------------+
|  g|  Gabby| 60|0.17073170731707318|
|  f|  Fanny| 36|0.34304852959857723|
|  d|  David| 29|0.34304852959857723|
|  e| Esther| 32|0.40545134654471543|
|  a|  Alice| 34|0.48915269944105694|
|  b|    Bob| 36|  2.514646227134146|
|  c|Charlie| 30| 2.7339209603658534|
+---+-------+---+-------------------+



#### (2) 动态PageRank,该算法一直运行，直到PR值收敛到预定于的公差值为止

In [18]:
graph.pageRank(tol=0.01).vertices.orderBy('pagerank').show()     # 设置公差值为0.0.1

+---+-------+---+-------------------+
| id|   name|age|           pagerank|
+---+-------+---+-------------------+
|  g|  Gabby| 60| 0.1799821386239711|
|  d|  David| 29| 0.3283606792049851|
|  f|  Fanny| 36| 0.3283606792049851|
|  e| Esther| 32|0.37085233187676075|
|  a|  Alice| 34|0.44910633706538744|
|  b|    Bob| 36|  2.655507832863289|
|  c|Charlie| 30| 2.6878300011606218|
+---+-------+---+-------------------+



## 六、基于GraphFrames的网页排名
#### 数据集地址：http://snap.stanford.edu/data/web-Google.html

#### 方法一：自己写一个类似的算法

In [ ]:
googlePath="web-Google.txt"

googleWeblinks=spark.sparkContext.textFile(googlePath).filter(lambda x:"#" not in x).map(lambda x:x.split("\t"))
# This parses the rows and filters out any comments, headers etc
def computeContribs(urls, rank):
    """Calculates URL contributions to the rank of other URLs."""
    num_urls = len(urls)
    contribs = []
    for url in urls:
        contribs.append((url, rank / num_urls))
    return contribs

In [ ]:
links = googleWeblinks.groupByKey().cache()

In [ ]:
ranks = links.map(lambda x: (x[0], 1.0)) # Initialized the ranks to 1 

In [ ]:
for iteration in range(2):
    contribs = links.join(ranks).flatMap(lambda x:computeContribs(x[1][0], x[1][1]))
    ranks = contribs.reduceByKey(lambda x,y:x+y).mapValues(lambda rank: rank * 0.85 + 0.15)
    
for (link,rank) in ranks.sortBy(lambda x:-x[1]).take(10):
    print("%s has rank: %s." % (link, rank))

#### 方法二：利用GraphFrame自带的PageRank算法

In [ ]:
from pyspark.sql.types import StructField, StructType, LongType
schema = StructType([StructField('src', LongType(), True), StructField('dst', LongType(), True)])
edgesDF = spark.read.load('web-Google.txt', format='csv', delimiter = '\t', schema = schema, mode = 'DROPMALFORMED')
edgesDF.show()

In [ ]:
edgesDF.cache()

In [ ]:
srcDF = edgesDF.select(edgesDF.src).distinct()
distDF = edgesDF.select(edgesDF.dst).distinct()
verticesDF = srcDF.union(distDF).distinct().withColumnRenamed('src', 'id')

In [ ]:
verticesDF.cache()

In [ ]:
from graphframes import GraphFrame
graph = GraphFrame(verticesDF, edgesDF)
ranks = graph.pageRank(resetProbability=0.15, maxIter = 5)
ranks.show()